In [1]:
load_model = 'inception_v3_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image

import pandas as pd

# sklearn metrics
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score


## load the inception_v3 based model

In [3]:
inception_v3_loaded = models.inception_v3(pretrained=True)

for param in inception_v3_loaded.parameters():
    param.requires_grad = False

inception_v3_loaded.fc = nn.Linear(in_features=2048, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inception_v3_loaded = inception_v3_loaded.to(device)



inception_v3_loaded.load_state_dict(torch.load('inception_v3_based_model2.pt'))
inception_v3_loaded.eval()

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/rka73/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [4]:
model = inception_v3_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [6]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

y_true = []
y_pred = []

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]
            y_true.append(breed)
            y_pred.append(predicted_breed)

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

In [7]:
df = pd.DataFrame(list(zip(y_true, y_pred)),
               columns =['Actual_breed', 'Predicted_breed'])

df

,Actual_breed,Predicted_breed
0,Pomeranian,Pomeranian
1,Pomeranian,Pomeranian
2,Pomeranian,Pomeranian
3,Pomeranian,Pomeranian
4,Pomeranian,Pomeranian
...,...,...
2105,papillon,Shih Tzu
2106,papillon,papillon
2107,papillon,papillon
2108,papillon,papillon


## Confusion Matrix

In [8]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


[[22  0  0 ...  0  0  0]
 [ 0 16  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ... 14  0  0]
 [ 1  0  0 ...  0 12  0]
 [ 0  0  1 ...  0  0 12]]


## Precision, Recall and F1

In [9]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

                                precision    recall  f1-score   support

                  Afghan hound      0.815     0.917     0.863        24
           African hunting dog      0.941     0.941     0.941        17
                      Airedale      0.600     1.000     0.750        21
American Staffordshire terrier      0.667     0.706     0.686        17
                   Appenzeller      0.714     0.312     0.435        16
            Australian terrier      0.882     0.750     0.811        20
            Bedlington terrier      0.792     1.000     0.884        19
          Bernese mountain dog      0.677     0.955     0.792        22
              Blenheim spaniel      1.000     0.947     0.973        19
                 Border collie      0.625     0.667     0.645        15
                Border terrier      0.773     0.944     0.850        18
                   Boston bull      0.895     0.895     0.895        19
          Bouvier des Flandres      0.818     0.600     0.692  

## Macro F1 Score

In [10]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred, average='macro')


0.8017306869182433

## Micro F1 Score

In [11]:
f1_score(y_true, y_pred, average='micro')


0.8085308056872038

## Weighted F1 Score

In [12]:
f1_score(y_true, y_pred, average='weighted')

0.80446386102536

## Cohen Kappa Score

In [13]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_true, y_pred)

0.8068829714522359

## Total Accuracy calculated from per class accuracy

In [14]:
# calculated from per class accuracy
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

80.38269624395788

## Per Breed Accuracy

### Top 10 Breeds with least accuracy

In [15]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 6.666666666666667),
 ('Appenzeller', 31.25),
 ('standard schnauzer', 31.25),
 ('Lakeland terrier', 35.0),
 ('Walker hound', 43.75),
 ('miniature poodle', 50.0),
 ('Staffordshire bullterrier', 56.25),
 ('collie', 56.25),
 ('Lhasa', 57.89473684210527),
 ('Bouvier des Flandres', 60.0)]

### Top 10 Breeds with most accuracy

In [16]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('schipperke', 100.0),
 ('Old English sheepdog', 100.0),
 ('miniature schnauzer', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0),
 ('Rottweiler', 100.0),
 ('Kerry blue terrier', 100.0),
 ('groenendael', 100.0),
 ('basenji', 100.0)]